# BERT NE recognition FACILITY

## 1)  Preprocess / format data

In [1]:
from __future__ import absolute_import, division, print_function

In [2]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re
import keras
import nltk
import spacy

Using TensorFlow backend.


In [3]:
from nltk import ne_chunk, pos_tag
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer
from nltk.corpus import stopwords
import gensim.models.word2vec as w2v
from gensim import logging
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as  pd
import seaborn as sns
import pickle
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import json
from nltk.tokenize import WhitespaceTokenizer
from nltk.tokenize import MWETokenizer

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [6]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/chantana/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## 2) load keyword: LOC from text

In [7]:
word_files = ['./keywords/list_hotel.txt']
word_list_name = []
for word_file in word_files:
    with open(word_file) as f:
        lines = f.readlines()
    word_list2 = [x.strip() for x in lines] 
    
    word_list_name.extend(word_list2)
word_list_name[0:5]

word_hotels = ['./keywords/list_location.txt']
lists_location = []
for word_hotel in word_hotels:
    with open(word_hotel) as f:
        lines = f.readlines()
    lists_l= [x.strip() for x in lines] 
    
    lists_location.extend(lists_l)
lists_location[0:5]

['99/89 Moo 6',
 'VIPHAVADEE RANDSIT RD',
 '111/1 Soi Ngam Wong Wan 47 Yaek 1',
 '310/72 Soi Phaholyothin 67/1',
 '3/2899 Soi Yak']

In [8]:
#remove duplicates
def ordered_set(in_list):
    out_list = []
    added = set()
    for val in in_list:
        if not val in added:
            out_list.append(val)
            added.add(val)
    return out_list

In [9]:
word_list_name = ordered_set(word_list_name)
word_location= ordered_set(lists_location)

In [10]:
mw_list = []

     
for x in word_list_name:
    ws = x.split() 
    if len(ws) > 1:
        tws = tuple(ws)
        mw_list.append(tws)
        

for x in word_location:
    ws = x.split() 
    if len(ws) > 1:
        tws = tuple(ws)
        mw_list.append(tws)
        

            
mw_list = ordered_set(mw_list)
mw_list[0:5]

[('Petpimarn', 'Boutique', 'Resort'),
 ('Airport', 'Suite', 'Bangkok', '(Don', 'Muang', 'Airport)'),
 ('The', 'Riche', 'Residence'),
 ('Regent', 'Home1', 'at', 'Donmuang'),
 ('Charoenpong', 'Apartment')]

In [11]:
my_idx = {}
for i,w in enumerate(mw_list):
    my_idx[" ".join(w)] = i
print(len(my_idx))

35803


In [12]:
mwe = MWETokenizer(mw_list,separator=' ')

In [13]:
reviews = pd.read_csv("./keywords/location.csv")

In [14]:
reviews.shape

(13019, 1)

In [15]:
reviews.head()

,location
0,Staying at @ Home Executive Apartment is a goo...
1,Staying at @ Oasis Resort is a good choice whe...
2,Staying at @ T Boutique Hotel is a good choice...
3,Staying at AC Resort is a good choice when you...
4,Staying at Acqua Condominium is a good choice ...


In [16]:
reviews = reviews.dropna()
reviews = reviews.reset_index(drop=True)

In [17]:
def spans(txt):
    tokens=mwe.tokenize(word_tokenize(txt))
    offset = 0
    for token in tokens:
        offset = txt.find(token, offset)
        yield token, offset, offset+len(token)
        offset += len(token)

In [78]:
from tqdm import tqdm_notebook as tqdm
all_item = []

for i in tqdm(range(len(reviews))):
    word_ls = []
    #print("sentence:",i+1)
    
    for token in spans(reviews.location[i]):
        #print(token)
        #assert token[0]==reviews.location[i][token[1]:token[2]]
        my_tuple = token[0]
        #print("###",token[0].lower())
        
        #my_tuples = ' , '.join(map(str, my_tuple))
        
         
        if token[0] in word_location:
            #word_ls.append(my_tuple)
            subwords = token[0].split()
            pos_list = [nltk.pos_tag([w]) for w in subwords]
            tag_list = ['I-LOC']*len(pos_list)
            tag_list[0] = 'B-LOC' 
            
            
            for s,p,t in zip(subwords,pos_list,tag_list):           
                if type(p) == list:
                    p = p[0][1]
                    #print('list')
                new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s,'POS': p})
                all_item.append(new_item)
                 
                
            #lis_lo = nltk.pos_tag(word_ls),LOC
            #print(' , '.join(map(str, lis_lo)))
 
        elif token[0] in word_list_name:
            
            subwords = token[0].split()
            pos_list = [nltk.pos_tag([w]) for w in subwords]
            tag_list = ['I-ORG']*len(pos_list)
            tag_list[0] = 'B-ORG' 
            
            #print(subwords)
            for s,p,t in zip(subwords,pos_list,tag_list):           
                new_item = dict({'Sentence #': i+1, 'Tag' : t, 'Word': s,'POS': p[0][1]})
                all_item.append(new_item)
                #print(new_item)
            #word_ls.append(my_tuple)
            #print("found LOC")
            #lis_lo = nltk.pos_tag(word_ls),"LOC"
            #print(' , '.join(map(str, lis_lo)))
                 

        else:
            #print(type(my_tuple))
            my_pos = nltk.pos_tag([my_tuple])[0][1]
            new_item = dict({'Sentence #': i+1, 'Tag' : 'O', 'Word': my_tuple.lower(),'POS': my_pos})
            all_item.append(new_item)
            
            #if i+1 == 5177:
                #print(new_item)
             
            #print(nltk.pos_tag([my_tuple]),',','O')
        #print(new_item)
        #if not(my_pos == '.' or  my_pos == ',' or my_pos == ':' or my_pos == '(' or my_pos == ')') :
            
            #all_item.append(new_item) 

        

        
    

          

## 3) save preproc data

In [79]:
file_csv = 'data-loc-bert.csv'
with open(file_csv, 'w') as csv_file:
    csv_file.write('Sentence #*Word*POS*Tag\n')
    for item in all_item:
        
        #print(item['POS'])    
        csv_file.write(str(item['Sentence #'])+'*'+item['Word']+'*'+item['POS']+'*'+item['Tag']+'\n')
csv_file.close()

## 4) Load preprocessed data

In [60]:
import pandas as pd
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
#load data
# explore data
file_csv = 'data-loc-bert.csv'
data = pd.read_csv(file_csv, sep='*',encoding="utf-8").fillna(method="ffill")

data.head(25)
 


,Sentence #,Word,POS,Tag
0,1,staying,VBG,O
1,1,at,IN,O
2,1,@,NN,B-ORG
3,1,Home,NN,I-ORG
4,1,Executive,NN,I-ORG
5,1,Apartment,NN,I-ORG
6,1,is,VBZ,O
7,1,a,DT,O
8,1,good,JJ,O
9,1,choice,NN,O


### 5) BERT trained-formating

In [27]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [28]:
 
#concat sentence
getter = SentenceGetter(data)

In [61]:
word_list = [ [s[0] for s in sent] for sent in getter.sentences] 

In [62]:
word_list[0]

[u'staying',
 u'at',
 u'@',
 u'Home',
 u'Executive',
 u'Apartment',
 u'is',
 u'a',
 u'good',
 u'choice',
 u'when',
 u'you',
 u'are',
 u'visiting',
 u'Central',
 u'Pattaya',
 u'.']

In [63]:
#sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences = word_list
sentences[0]

[u'staying',
 u'at',
 u'@',
 u'Home',
 u'Executive',
 u'Apartment',
 u'is',
 u'a',
 u'good',
 u'choice',
 u'when',
 u'you',
 u'are',
 u'visiting',
 u'Central',
 u'Pattaya',
 u'.']

In [64]:
print(len(sentences))

13019


In [65]:
labels = [[s[2] for s in sent] for sent in getter.sentences]
print(len(labels[0]))

17


In [33]:
print(len(labels))

13019


In [67]:
print(len(word_list[0]))

17


In [68]:
tags_vals = list(set(data["Tag"].values))
tag2idx = {t: i for i, t in enumerate(tags_vals)}
idx2tag = {i: t for i, t in enumerate(tags_vals) }

In [69]:
words = list(set(data["Word"].values))
n_words = len(words); n_words

14126

In [78]:
word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1
word2idx["PAD"] = 0
idx2word = {i: w for w, i in word2idx.items()}
 
 

# Training

In [71]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

In [142]:
MAX_LEN = 128
bs = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0) 
torch.cuda.set_device(1)


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

mwe = MWETokenizer(mw_list,separator=' ')

tokenized_texts = [mwe.tokenize(word_tokenize(sent)) for sent in sentences]



array = []
for text in tokenized_texts:
    ll = []
    
    for t in text:
        
        tt = t.lower().split()
        #print(tt)
        ll.extend(tt)
    array.append(ll)
print(array[0]) 

In [143]:
print(tokenized_texts[0],len(tokenized_texts[0]))

[u'staying', u'at', u'@', u'Home', u'Executive', u'Apartment', u'is', u'a', u'good', u'choice', u'when', u'you', u'are', u'visiting', u'Central', u'Pattaya', u'.'] 17


In [144]:
tokenized_texts = word_list

tokenized_texts = [ tokenizer.tokenize(sent)   for sent in sentences]
print(tokenized_texts[0])

In [145]:
sentences[0]

[u'staying',
 u'at',
 u'@',
 u'Home',
 u'Executive',
 u'Apartment',
 u'is',
 u'a',
 u'good',
 u'choice',
 u'when',
 u'you',
 u'are',
 u'visiting',
 u'Central',
 u'Pattaya',
 u'.']

In [80]:

for s in tokenized_texts:
    for w in s:
        if w not in word2idx:
            print(s)
                 


In [87]:
tokens_ids = [[word2idx[w] for w in s] for s in tokenized_texts]
print(len(tokens_ids[0]),len(labels[0]))


17 17


In [88]:
#input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
input_ids = pad_sequences(tokens_ids,
                          maxlen=MAX_LEN, dtype="int64", truncating="post", padding="post")
#input_ids = pad_sequences([convert_word2idx(txt) for txt in tokenized_texts],
                          #maxlen=MAX_LEN, dtype="int64", truncating="post", padding="post")

In [49]:
#input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="int64", truncating="post", padding="post")


In [89]:
print(MAX_LEN)
for i in tokens_ids:
    if len(i) > MAX_LEN:
        #print(tokens_ids)
        print("more")

128


In [90]:
t_list = [[tag2idx.get(l) for l in lab] for lab in labels]
len(t_list[0])

17

In [91]:
print(input_ids[0])

[ 8079  1669 10542  3537  3395   532  7601  5382  3697   900 11385  5139
 12844  9264  7504 10694   555     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]


In [92]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="int64", truncating="post")

In [93]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [110]:
#split train test
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.3)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.3)

In [111]:
#change to torhc tensor
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [112]:
print(tr_inputs.shape)
print(tr_masks.shape)
print(tr_tags.shape)

torch.Size([9113, 128])
torch.Size([9113, 128])
torch.Size([9113, 128])


In [113]:
print(val_inputs.shape)
print(val_masks.shape)
print(val_tags.shape)

torch.Size([3906, 128])
torch.Size([3906, 128])
torch.Size([3906, 128])


In [114]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [115]:
print(len(tag2idx))

5


In [100]:
#torch.cuda.empty_cache()

In [116]:
model = BertForTokenClassification.from_pretrained(u"bert-base-uncased", num_labels=len(tag2idx))

2019-03-22 18:35:34,285 : INFO : loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /home/chantana/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
2019-03-22 18:35:34,288 : INFO : extracting archive file /home/chantana/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpEFKcLd
2019-03-22 18:35:37,928 : INFO : Model config {
  "attention_probs_dropout_prob": 0.1, 
  "hidden_act": "gelu", 
  "hidden_dropout_prob": 0.1, 
  "hidden_size": 768, 
  "initializer_range": 0.02, 
  "intermediate_size": 3072, 
  "max_position_embeddings": 512, 
  "num_attention_heads": 12, 
  "num_hidden_layers": 12, 
  "type_vocab_size": 2, 
  "vocab_size": 30522
}

2019-03-22 18:35:40,961 : INFO : Weights of BertForTokenClass

In [117]:
model.cuda();

In [118]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [119]:
#Fine tune BERT
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat) 

In [120]:
epochs = 10
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags_vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags_vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))



Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Train loss: 0.159840503954
Validation loss: 0.0273056180482
Validation Accuracy: 0.661922034439




Epoch:  10%|█         | 1/10 [02:44<24:44, 164.94s/it]

F1-Score: 0.258531979911
Train loss: 0.0198791270557
Validation loss: 0.0153636987883
Validation Accuracy: 0.751247608418




Epoch:  20%|██        | 2/10 [05:31<22:03, 165.39s/it]

...

F1-Score: 0.308567096285
Train loss: 0.00179917830271
Validation loss: 0.0150483870009
Validation Accuracy: 0.699848533163




Epoch:  90%|█████████ | 9/10 [25:12<02:48, 168.95s/it]

F1-Score: 0.367452192689
Train loss: 0.00146136706771
Validation loss: 0.0165137722949
Validation Accuracy: 0.717817283163




Epoch: 100%|██████████| 10/10 [28:01<00:00, 169.21s/it]

F1-Score: 0.346016179723


In [121]:
#evaluate model
model.eval()
predictions = []
true_labels = []
true_inputs = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
print(len(valid_dataloader))
for batch in tqdm(valid_dataloader):
    #print(len(batch))
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    inputs = b_input_ids.to('cpu').numpy()
    true_inputs.append(inputs)
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))



  0%|          | 0/245 [00:00<?, ?it/s]

245




  0%|          | 1/245 [00:00<00:35,  6.96it/s]

  1%|          | 2/245 [00:00<00:34,  7.01it/s]

  1%|          | 3/245 [00:00<00:33,  7.15it/s]

  2%|▏         | 4/245 [00:00<00:32,  7.31it/s]

  2%|▏         | 5/245 [00:00<00:32,  7.37it/s]

  2%|▏         | 6/245 [00:00<00:32,  7.44it/s]

  3%|▎         | 7/245 [00:00<00:31,  7.47it/s]

  3%|▎         | 8/245 [00:01<00:31,  7.48it/s]

  4%|▎         | 9/245 [00:01<00:31,  7.60it/s]

  4%|▍         | 10/245 [00:01<00:30,  7.69it/s]

  4%|▍         | 11/245 [00:01<00:30,  7.74it/s]

  5%|▍         | 12/245 [00:01<00:30,  7.70it/s]

  5%|▌         | 13/245 [00:01<00:30,  7.62it/s]

  6%|▌         | 14/245 [00:01<00:30,  7.59it/s]

  6%|▌         | 15/245 [00:01<00:30,  7.58it/s]

  7%|▋         | 16/245 [00:02<00:30,  7.57it/s]

  7%|▋         | 17/245 [00:02<00:30,  7.56it/s]

  7%|▋         | 18/245 [00:02<00:30,  7.55it/s]

  8%|▊         | 19/245 [00:02<00:29,  7.54it/s]

  8%|▊         | 20/245 [00:02<00:29,  7.60it/s]

  9%|▊ 

Validation loss: 0.0165137722949
Validation Accuracy: 0.717817283163
Validation F1-Score: 0.346016179723


In [136]:
#evaluate model
model.eval()
predictions = []
true_labels = []
true_inputs = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
print(len(valid_dataloader))
for batch in tqdm(valid_dataloader):
    #print(len(batch))
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    
    inputs = b_input_ids.to('cpu').numpy()
    true_inputs.append(inputs)
    true_labels.append(label_ids)
    #print(inputs)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags_vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags_vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
valid_inputs = [[idx2word[l_ii] for l_ii in l_i] for l in  true_inputs  for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))




  0%|          | 0/245 [00:00<?, ?it/s]


  0%|          | 1/245 [00:00<00:35,  6.81it/s]

245





  1%|          | 2/245 [00:00<00:35,  6.94it/s]


  1%|          | 3/245 [00:00<00:33,  7.16it/s]


  2%|▏         | 4/245 [00:00<00:32,  7.34it/s]


  2%|▏         | 5/245 [00:00<00:32,  7.47it/s]


  2%|▏         | 6/245 [00:00<00:31,  7.58it/s]


  3%|▎         | 7/245 [00:00<00:31,  7.56it/s]


  3%|▎         | 8/245 [00:01<00:31,  7.64it/s]


  4%|▎         | 9/245 [00:01<00:31,  7.61it/s]


  4%|▍         | 10/245 [00:01<00:30,  7.59it/s]


  4%|▍         | 11/245 [00:01<00:30,  7.63it/s]


  5%|▍         | 12/245 [00:01<00:30,  7.68it/s]


  5%|▌         | 13/245 [00:01<00:30,  7.71it/s]


  6%|▌         | 14/245 [00:01<00:29,  7.73it/s]


  6%|▌         | 15/245 [00:01<00:29,  7.78it/s]


  7%|▋         | 16/245 [00:02<00:29,  7.87it/s]


  7%|▋         | 17/245 [00:02<00:28,  7.95it/s]


  7%|▋         | 18/245 [00:02<00:28,  7.96it/s]


  8%|▊         | 19/245 [00:02<00:28,  8.01it/s]


  8%|▊         | 20/245 [00:02<00:28,  7.91it/s]


  9%|▊         | 21/245 [00:02<00:28

 56%|█████▋    | 138/245 [00:17<00:13,  7.92it/s]


 57%|█████▋    | 139/245 [00:17<00:13,  7.96it/s]


 57%|█████▋    | 140/245 [00:17<00:13,  7.99it/s]


 58%|█████▊    | 141/245 [00:18<00:12,  8.02it/s]


 58%|█████▊    | 142/245 [00:18<00:12,  8.02it/s]


 58%|█████▊    | 143/245 [00:18<00:12,  8.03it/s]


 59%|█████▉    | 144/245 [00:18<00:12,  7.92it/s]


 59%|█████▉    | 145/245 [00:18<00:12,  7.87it/s]


 60%|█████▉    | 146/245 [00:18<00:12,  7.87it/s]


 60%|██████    | 147/245 [00:18<00:12,  7.89it/s]


 60%|██████    | 148/245 [00:18<00:12,  7.94it/s]


 61%|██████    | 149/245 [00:19<00:12,  7.88it/s]


 61%|██████    | 150/245 [00:19<00:12,  7.81it/s]


 62%|██████▏   | 151/245 [00:19<00:12,  7.79it/s]


 62%|██████▏   | 152/245 [00:19<00:11,  7.82it/s]


 62%|██████▏   | 153/245 [00:19<00:11,  7.89it/s]


 63%|██████▎   | 154/245 [00:19<00:11,  7.94it/s]


 63%|██████▎   | 155/245 [00:19<00:11,  7.98it/s]


 64%|██████▎   | 156/245 [00:19<00:11,  8.00it/s]


 64%|██████▍

Validation loss: 0.0165137722949
Validation Accuracy: 0.717817283163
Validation F1-Score: 0.346016179723


In [141]:
for i,j,ll in zip(pred_tags,valid_tags,val_inputs):
    for k,l,kk in zip(i,j,ll):
        if k==l and k != 'O' : 
            print(k,l,idx2word[kk.item()])

B-ORG B-ORG Laguna
I-ORG I-ORG Beach
I-ORG I-ORG Resort
I-ORG I-ORG Jomtien1
I-ORG I-ORG Condominium
B-LOC B-LOC Jomtien
I-LOC I-LOC Beach
B-LOC B-LOC Pattaya
B-LOC B-LOC Tinnies
I-LOC I-LOC Thailand
B-LOC B-LOC Surf
I-LOC I-LOC Kitchen
B-LOC B-LOC Toto
I-LOC I-LOC Italian
I-LOC I-LOC Restaurant
I-LOC I-LOC and
I-LOC I-LOC Pizzeria
B-LOC B-LOC Jomtien
I-LOC I-LOC Beach
B-LOC B-LOC Sugar
I-LOC I-LOC Shack
I-LOC I-LOC Bar
B-LOC B-LOC Utapao
I-LOC I-LOC Airport
B-ORG B-ORG Cham
I-ORG I-ORG Cha
I-ORG I-ORG Hostel
B-LOC B-LOC Phra
I-LOC I-LOC Sing
B-LOC B-LOC Chiang
I-LOC I-LOC Mai
I-LOC I-LOC International
I-LOC I-LOC Airport
B-ORG B-ORG Siam
I-ORG I-ORG Piman
I-ORG I-ORG Hotel
B-LOC B-LOC Bangkok
B-LOC B-LOC El
I-LOC I-LOC Sabor
I-LOC I-LOC de
I-LOC I-LOC Minburi
B-LOC B-LOC Tummour
B-LOC B-LOC Pang
I-LOC I-LOC Pa
B-LOC B-LOC Suvarnabhumi
I-LOC I-LOC Intl
B-LOC B-LOC Don
I-LOC I-LOC Muang
I-LOC I-LOC Airport
B-ORG B-ORG Bloo
I-ORG I-ORG Hostel
B-LOC B-LOC Phuket
I-LOC I-LOC Town
B-ORG B-O

I-LOC I-LOC Revolution
B-LOC B-LOC Suvarnabhumi
I-LOC I-LOC Intl
B-LOC B-LOC Don
I-LOC I-LOC Muang
I-LOC I-LOC Airport
B-LOC B-LOC Thong
I-LOC I-LOC Lo
B-LOC B-LOC Phrom
I-LOC I-LOC Phong
B-ORG B-ORG Hueandin
I-ORG I-ORG Klinmai
I-ORG I-ORG Boutique
I-ORG I-ORG Hotel
I-ORG I-ORG Lodge
B-LOC B-LOC Chiang
I-LOC I-LOC Mai
B-LOC B-LOC Paak
I-LOC I-LOC Dang
I-LOC I-LOC Riverside
I-LOC I-LOC Dining
B-LOC B-LOC Fa
I-LOC I-LOC Ham
B-LOC B-LOC Oxygen
I-LOC I-LOC Dining
I-LOC I-LOC room
B-LOC B-LOC CentralFestival
I-LOC I-LOC Chiangmai
B-LOC B-LOC Wat
I-LOC I-LOC Gate
B-LOC B-LOC Chiang
I-LOC I-LOC Mai
I-LOC I-LOC Intl
I-LOC I-LOC Airport
B-LOC B-LOC Lampang
I-LOC I-LOC Airport
B-ORG B-ORG White
I-ORG I-ORG Elephant
I-ORG I-ORG Krabi
I-ORG I-ORG Lodge
B-LOC B-LOC Sai
I-LOC I-LOC Thai
B-LOC B-LOC Bistro
I-LOC I-LOC Monaco
B-LOC B-LOC Maharat
I-LOC I-LOC Bakery
I-LOC I-LOC and
I-LOC I-LOC Restaurant
B-LOC B-LOC Gecko
I-LOC I-LOC Cabane
I-LOC I-LOC Restaurant
B-LOC B-LOC Krabi
B-LOC B-LOC Krabi
I-L

I-ORG I-ORG Miami
I-ORG I-ORG Bungalows
I-ORG I-ORG Ranch
B-LOC B-LOC Ko
I-LOC I-LOC Lanta
B-LOC B-LOC Koh
I-LOC I-LOC Lanta
B-ORG B-ORG SriLanta
I-ORG I-ORG Resort
B-LOC B-LOC Koh
I-LOC I-LOC Lanta
B-LOC B-LOC Horizon
B-LOC B-LOC Krabi
I-LOC I-LOC Airport
B-LOC B-LOC Trang
I-LOC I-LOC Airport
B-ORG B-ORG Baan
I-ORG I-ORG Pra
I-ORG I-ORG Nond
I-ORG I-ORG Bed
I-ORG I-ORG &
I-ORG I-ORG Breakfast
B-LOC B-LOC Yan
I-LOC I-LOC Nawa
B-LOC B-LOC Surasak
I-LOC I-LOC BTS
I-LOC I-LOC Station
B-ORG B-ORG Baan
I-ORG I-ORG Pra
I-ORG I-ORG Nond
I-ORG I-ORG Bed
I-ORG I-ORG &
I-ORG I-ORG Breakfast
B-LOC B-LOC Surasak
I-LOC I-LOC BTS
I-LOC I-LOC Station
B-LOC B-LOC Lerdsin
I-LOC I-LOC Hospital
B-ORG B-ORG B2
I-ORG I-ORG Jomtien
I-ORG I-ORG Pattaya
I-ORG I-ORG Boutique
I-ORG I-ORG &
I-ORG I-ORG Budget
I-ORG I-ORG Hotel
B-LOC B-LOC Na
I-LOC I-LOC Jomtien
B-ORG B-ORG Eight
I-ORG I-ORG Inn
I-ORG I-ORG Hotel
B-LOC B-LOC Bangkok
B-LOC B-LOC BarSu
B-LOC B-LOC amBar
B-LOC B-LOC Sunrise
I-LOC I-LOC Tacos
I-LOC I

I-LOC I-LOC Ice
I-LOC I-LOC Cream
B-LOC B-LOC My
I-LOC I-LOC Friend
I-LOC I-LOC Thai
I-LOC I-LOC Restaurant
B-LOC B-LOC Na
I-LOC I-LOC Suang
B-LOC B-LOC Tsunami
I-LOC I-LOC Tailor
B-LOC B-LOC Mr
I-LOC I-LOC Tailor
I-LOC I-LOC Khao
I-LOC I-LOC Lak
B-LOC B-LOC Phuket
I-LOC I-LOC Intl
I-LOC I-LOC Airport
B-ORG B-ORG Bora
I-ORG I-ORG Bora
I-ORG I-ORG Villas
I-ORG I-ORG Phuket
B-LOC B-LOC Chalong
B-LOC B-LOC nearby
B-LOC B-LOC Phunaka
I-LOC I-LOC Golf
I-LOC I-LOC Course
B-LOC B-LOC Chalong
I-LOC I-LOC Temple
B-ORG B-ORG Thai
I-ORG I-ORG Crystal
I-ORG I-ORG Resident
B-ORG B-ORG Thai
I-ORG I-ORG Crystal
I-ORG I-ORG Resident
B-LOC B-LOC Crystal
I-LOC I-LOC Bay
I-LOC I-LOC Golf
I-LOC I-LOC Club
B-LOC B-LOC Bangpra
I-LOC I-LOC International
I-LOC I-LOC Golf
I-LOC I-LOC Club
B-ORG B-ORG Malee
I-ORG I-ORG Beach
I-ORG I-ORG Guest
I-ORG I-ORG House
B-LOC B-LOC Phra
I-LOC I-LOC Tamnak
I-LOC I-LOC Hill
B-LOC B-LOC nearby
B-LOC B-LOC Laem
I-LOC I-LOC Bali
I-LOC I-LOC Hai
B-LOC B-LOC Pattaya
I-LOC I-LOC

I-ORG I-ORG Marriott
I-ORG I-ORG Hotel
I-ORG I-ORG Bangkok
B-ORG B-ORG JW
I-ORG I-ORG Marriott
I-ORG I-ORG Hotel
I-ORG I-ORG Bangkok
B-LOC B-LOC Don
I-LOC I-LOC Muang
I-LOC I-LOC Airport
B-LOC B-LOC Suvarnabhumi
I-LOC I-LOC Intl
B-LOC B-LOC Ploen
I-LOC I-LOC Chit
B-LOC B-LOC Nana
B-LOC B-LOC Khao
I-LOC I-LOC Takiab
B-LOC B-LOC Hua
I-LOC I-LOC Hin
I-LOC I-LOC Railway
I-LOC I-LOC Station
B-LOC B-LOC nearby
B-LOC B-LOC Sea
I-LOC I-LOC Pines
I-LOC I-LOC Golf
I-LOC I-LOC Course
B-ORG B-ORG Jirapa
I-ORG I-ORG Modern
I-ORG I-ORG Resort
B-LOC B-LOC Bo
I-LOC I-LOC Win
B-ORG B-ORG We
I-ORG I-ORG Inn
I-ORG I-ORG Lodge
B-LOC B-LOC Bangkok
B-LOC B-LOC Lancake
B-LOC B-LOC Cho
I-LOC I-LOC Khao
I-LOC I-LOC Mu
I-LOC I-LOC Daeng
I-LOC I-LOC Mu
I-LOC I-LOC Krop
I-LOC I-LOC Restaurant
B-LOC B-LOC 2
B-LOC B-LOC Coffee
I-LOC I-LOC House
B-LOC B-LOC Siam
I-LOC I-LOC Park
I-LOC I-LOC City
B-LOC B-LOC Suvarnabhumi
I-LOC I-LOC Intl
B-LOC B-LOC Don
I-LOC I-LOC Muang
I-LOC I-LOC Airport
B-ORG B-ORG Well
I-ORG I-O

...

I-ORG I-ORG Hotel
B-LOC B-LOC Bangkok
B-LOC B-LOC Konnichipan
B-LOC B-LOC Juntun
B-LOC B-LOC Snooze
I-LOC I-LOC Coffee
I-LOC I-LOC House
B-LOC B-LOC Jok
I-LOC I-LOC Pochana
B-LOC B-LOC Don
I-LOC I-LOC Muang
I-LOC I-LOC Airport
B-LOC B-LOC Suvarnabhumi
I-LOC I-LOC Intl
B-ORG B-ORG Pongsakorn
I-ORG I-ORG Boutique
I-ORG I-ORG Resort
B-LOC B-LOC Suvarnabhumi
I-LOC I-LOC International
I-LOC I-LOC Airport
B-LOC B-LOC nearby
B-LOC B-LOC Suvarnabhumi
I-LOC I-LOC International
I-LOC I-LOC Airport
B-ORG B-ORG Peace
I-ORG I-ORG Blue
I-ORG I-ORG Naturist
I-ORG I-ORG Resort
I-ORG I-ORG Naiharn
I-ORG I-ORG Beach
I-ORG I-ORG Specialty
I-ORG I-ORG Resort
B-LOC B-LOC Rawai
B-LOC B-LOC Neko
I-LOC I-LOC Cat
I-LOC I-LOC Cafe
I-LOC I-LOC Phuket
B-LOC B-LOC Zen
I-LOC I-LOC Eat
I-LOC I-LOC Phuket
B-LOC B-LOC A
I-LOC I-LOC Spoonful
I-LOC I-LOC of
I-LOC I-LOC Sugar
B-LOC B-LOC Nai
I-LOC I-LOC Harn
B-LOC B-LOC Phuket
I-LOC I-LOC Intl
I-LOC I-LOC Airport
B-LOC B-LOC Krabi
I-LOC I-LOC Airport
B-ORG B-ORG Pink
I-O

I-LOC I-LOC Thailand
B-LOC B-LOC Chiang
I-LOC I-LOC Mai
I-LOC I-LOC Intl
I-LOC I-LOC Airport
B-LOC B-LOC Lampang
I-LOC I-LOC Airport
B-ORG B-ORG Laguna
I-ORG I-ORG Villas
I-ORG I-ORG Resort
B-LOC B-LOC Thailand
B-LOC B-LOC L'eKrin
I-LOC I-LOC Restaurant
B-LOC B-LOC The
I-LOC I-LOC Terrace
I-LOC I-LOC House
I-LOC I-LOC Restaurant
B-LOC B-LOC Breeze
I-LOC I-LOC Beach
I-LOC I-LOC Bar
B-LOC B-LOC Jun
I-LOC I-LOC Hom
B-LOC B-LOC Chujai
I-LOC I-LOC Art
I-LOC I-LOC Gallery
B-LOC B-LOC Koh
I-LOC I-LOC Samui
I-LOC I-LOC Airport
B-ORG B-ORG BareFeet
I-ORG I-ORG Naturist
I-ORG I-ORG Resort
I-ORG I-ORG Specialty
I-ORG I-ORG Inn
B-LOC B-LOC Bangkok
B-LOC B-LOC Baan
I-LOC I-LOC Pla
I-LOC I-LOC Joom
B-LOC B-LOC Tiao
I-LOC I-LOC Ruea
I-LOC I-LOC Tor
I-LOC I-LOC Charm
B-LOC B-LOC Oishi
I-LOC I-LOC Shabushi
B-LOC B-LOC Krua
I-LOC I-LOC Sompong
B-LOC B-LOC Don
I-LOC I-LOC Muang
I-LOC I-LOC Airport
B-LOC B-LOC Suvarnabhumi
I-LOC I-LOC Intl
B-LOC B-LOC Central
I-LOC I-LOC Pattaya
B-LOC B-LOC Central
I-LOC 

B-LOC B-LOC Thai
I-LOC I-LOC Orchid
I-LOC I-LOC Village
I-LOC I-LOC Resort
B-LOC B-LOC Utapao
I-LOC I-LOC Airport
B-LOC B-LOC Suvarnabhumi
I-LOC I-LOC Intl
B-ORG B-ORG CHANN
I-ORG I-ORG Bangkok
I-ORG I-ORG Noi
I-ORG I-ORG Hotel
B-LOC B-LOC Konnichipan
B-LOC B-LOC Tom
I-LOC I-LOC Yum
I-LOC I-LOC Kung
B-LOC B-LOC PEEPS
I-LOC I-LOC Thai
I-LOC I-LOC Eatery
B-LOC B-LOC Don
I-LOC I-LOC Muang
I-LOC I-LOC Airport
B-LOC B-LOC Suvarnabhumi
I-LOC I-LOC Intl
B-ORG B-ORG Champagne
I-ORG I-ORG Villa
I-ORG I-ORG by
I-ORG I-ORG Jetta
B-LOC B-LOC Nai
I-LOC I-LOC Harn
B-ORG B-ORG The
I-ORG I-ORG i
I-ORG I-ORG Talay
I-ORG I-ORG Room
I-ORG I-ORG &
I-ORG I-ORG Souvenir
I-ORG I-ORG Guesthouse
I-ORG I-ORG Lodge
B-LOC B-LOC Krabi
B-LOC B-LOC Thailand
B-LOC B-LOC Bistro
I-LOC I-LOC Monaco
B-LOC B-LOC Gecko
I-LOC I-LOC Cabane
I-LOC I-LOC Restaurant
B-LOC B-LOC Krabi
B-LOC B-LOC Walking
I-LOC I-LOC Street
B-LOC B-LOC Krabi
I-LOC I-LOC Airport
B-LOC B-LOC Phuket
I-LOC I-LOC Intl
I-LOC I-LOC Airport
B-ORG B-ORG Th

B-LOC B-LOC Suvarnabhumi
I-LOC I-LOC Intl
B-ORG B-ORG I-Rovers
I-ORG I-ORG Luxury
I-ORG I-ORG Appartments
I-ORG I-ORG Condominium
B-LOC B-LOC Cafe
I-LOC I-LOC De
I-LOC I-LOC Beach
B-LOC B-LOC Jockys
I-LOC I-LOC Restaurant
B-LOC B-LOC Thai
I-LOC I-LOC Orchid
I-LOC I-LOC Village
I-LOC I-LOC Resort
B-LOC B-LOC Dongtan
I-LOC I-LOC Beach
B-LOC B-LOC Utapao
I-LOC I-LOC Airport
B-LOC B-LOC Suvarnabhumi
I-LOC I-LOC Intl
B-LOC B-LOC Pattaya
B-LOC B-LOC Moonlight
I-LOC I-LOC Fine
I-LOC I-LOC Dining
B-LOC B-LOC La
I-LOC I-LOC Bocca
I-LOC I-LOC Italian
I-LOC I-LOC Restaurant
I-LOC I-LOC and
I-LOC I-LOC Pizzeria
B-LOC B-LOC Tinnies
I-LOC I-LOC Thailand
B-LOC B-LOC Jomtien
I-LOC I-LOC Beach
B-LOC B-LOC Panorama
B-LOC B-LOC Utapao
I-LOC I-LOC Airport
B-ORG B-ORG Leaf
I-ORG I-ORG House
I-ORG I-ORG Bungalow
I-ORG I-ORG Hotel
B-LOC B-LOC Ko
I-LOC I-LOC Lanta
B-LOC B-LOC iRie
I-LOC I-LOC Bar
I-LOC I-LOC and
I-LOC I-LOC Restaurant
B-LOC B-LOC Somewhere
I-LOC I-LOC Else
B-LOC B-LOC Suza
I-LOC I-LOC Hut
B-L

I-ORG I-ORG Hotel
B-LOC B-LOC Thailand
B-LOC B-LOC The
I-LOC I-LOC Hungry
I-LOC I-LOC Wolf
B-LOC B-LOC Olive
I-LOC I-LOC Tree
I-LOC I-LOC Greek
I-LOC I-LOC Restaurant
B-LOC B-LOC Boss
I-LOC I-LOC International
I-LOC I-LOC Suits
B-LOC B-LOC Modern
I-LOC I-LOC Suit
I-LOC I-LOC Samui
B-LOC B-LOC G.Joseph
I-LOC I-LOC Custom
I-LOC I-LOC Tailor
B-LOC B-LOC Koh
I-LOC I-LOC Samui
I-LOC I-LOC Airport
B-ORG B-ORG Blue
I-ORG I-ORG Sky
I-ORG I-ORG House
I-ORG I-ORG Hotel
I-ORG I-ORG Kalim
B-LOC B-LOC Kalim
B-ORG B-ORG Mellow
I-ORG I-ORG Space
I-ORG I-ORG Boutique
I-ORG I-ORG Rooms
I-ORG I-ORG Hotel
B-LOC B-LOC Pad
I-LOC I-LOC Thai
I-LOC I-LOC Shop
B-LOC B-LOC Karon
I-LOC I-LOC Bistro
B-LOC B-LOC Ltd.
B-LOC B-LOC Phuket
I-LOC I-LOC Intl
I-LOC I-LOC Airport
B-LOC B-LOC Krabi
I-LOC I-LOC Airport
I-ORG I-ORG Park
I-ORG I-ORG Residence
B-LOC B-LOC Chonburi
B-LOC B-LOC Chonburi
B-LOC B-LOC Amata
I-LOC I-LOC Nakorn
I-LOC I-LOC Industrial
I-LOC I-LOC Park
B-LOC B-LOC Burapha
I-LOC I-LOC University
B-ORG B

In [ ]:
from tqdm import tqdm_notebook as tqdm

for lab in b_labels:
    for l in lab:
        print(idx2tag[int(l)])
        

In [384]:
[[ idx2tag[int(l)] for l in lab] for lab in b_labels]

[[u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O'],
 [u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'ORG',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',
  u'O',